## Extract FLASH Data in yt
David Schneidinger

9/26/24

Use yt's covering_grid class to extract data from FLASH.

In [3]:
# https://yt-project.org/

import yt
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [4]:
# plot_path = "./OSIRIS_transfer/MAGOFF/MagShockZ_hdf5_plt_cnt_0004"
plot_path = "~/cellar/OSIRIS_transfer/MAGON/MagShockZ_hdf5_chk_0005"
input_filename = "magshockz-v2.0.2d"
output_dir = Path(f"/home/dschneidinger/MagShockZ/input_files/{input_filename}")
if not output_dir.exists():
    output_dir.mkdir()

In [5]:
# Certain fields are not directly output by FLASH, but can be derived from the fields that are output.
# For example, the number density of electrons and each ion species
# Everything, with the exception of the thermal velocities (which are in osiris units), are in cgs

import sys
sys.path.append("../src")
from load_derived_FLASH_fields import derive_fields
ds = derive_fields(plot_path,rqm=100)

yt : [INFO     ] 2024-10-16 15:27:16,815 Particle file found: MagShockZ_hdf5_chk_0005


yt : [INFO     ] 2024-10-16 15:27:16,837 Parameters: current_time              = 1.8520020414484505e-09
yt : [INFO     ] 2024-10-16 15:27:16,837 Parameters: domain_dimensions         = [288 272 288]
yt : [INFO     ] 2024-10-16 15:27:16,837 Parameters: domain_left_edge          = [-0.6   -0.075 -0.6  ]
yt : [INFO     ] 2024-10-16 15:27:16,838 Parameters: domain_right_edge         = [0.6 1.  0.6]
yt : [INFO     ] 2024-10-16 15:27:16,838 Parameters: cosmological_simulation   = 0


In [10]:
# this is from the yt documentation

level = 2
dims = ds.domain_dimensions * ds.refine_by**level

# We construct an object that describes the data region and structure we want
# In this case, we want all data up to the maximum "level" of refinement
# across the entire simulation volume.  Higher levels than this will not
# contribute to our covering grid.

all_data = ds.covering_grid(
    level,
    left_edge=[-0.6, -0.075, -0.6],
    dims=dims,
    # And any fields to preload (this is optional!)
)
print(type(all_data))   

<class 'yt.data_objects.construction_data_containers.YTCoveringGrid'>


In [6]:
# All of these numbers come from the INITIAL conditions of FLASH, not the actual values at the given time


ne_cgs = 5e18 # [cm^-3]
e = 4.80320425e-10 # [statC] = [cm^3/2⋅g^1/2⋅s^−1]
m_e = 9.1093837139e-28 # [g]
c = 2.99792458e10 # [cm/s]
kb = 8.617333262e-5 # [eV/K]
proton_mass = 1.6726219e-24 # [g]
B = 150000 # [Gauss]
T_e = 40 # [eV]
Al_charge_state = 6

rqm = 100 # [m_e]

# from Wikipedia
aluminum_molecular_weight = 26.981 
magnesium_molecular_weight = 24.305


omega_pe = np.sqrt(4*np.pi*ne_cgs*e**2/m_e)
omega_pi = np.sqrt(4*np.pi*ne_cgs*Al_charge_state*e**2/(proton_mass*aluminum_molecular_weight)) # extra charge state factor got absorbed into ne
print(f"ion inertial length {c/omega_pi} cm")
omega_pi_osiris = np.sqrt(4*np.pi*ne_cgs*e**2/(rqm*m_e))

rho_0 = aluminum_molecular_weight*proton_mass*ne_cgs/Al_charge_state
alfven_speed = B/np.sqrt(4*np.pi*rho_0) # check this factor of 4pi
print(f"FLASH alfven speed: {np.format_float_scientific(alfven_speed,2)} cm/s")

print(f"alfven speed in ion inertial length/ion plasma-time {alfven_speed/(c/omega_pi)*omega_pi**-1}")
print(f"in osiris this would be {(alfven_speed/(c/omega_pi)*omega_pi**-1)*(omega_pe/omega_pi_osiris)**2}")

alfven_speed_osiris = (B/np.sqrt(4*np.pi*ne_cgs*rqm*m_e))/c
print(f"OSIRIS alfven speed: {alfven_speed_osiris,2} c")
print(f"OSIRIS alfven speed: {np.format_float_scientific(c*alfven_speed_osiris,2)} cm/s")


ion inertial length 0.021594939868701025 cm
FLASH alfven speed: 6.9e+06 cm/s
alfven speed in ion inertial length/ion plasma-time 0.00023015919863826265
in osiris this would be 0.023015919863826263
OSIRIS alfven speed: (0.0020913952814540443, 2) c
OSIRIS alfven speed: 6.27e+07 cm/s


In [7]:
# You need to clean up this code later

def save_slice(normalizations:dict, target_index:int=84):
    '''
    args: normalizations: dict, target_index: int
    normalizations: dict
        key: str, field name
        value: float, normalization factor. Note: function automatically divides by this factor
    target_index: int
        index of the edge of the target

    Note: Density data will be output as a numpy array because OSIRIS uses its own interpolator for density data
    '''

    # We are looking in the x-y plane at z = 0
    z_middle_index = dims[2] // 2

    # Normalize spatial dimensions to electron inertial length
    x = all_data['flash','x'][:,0,0]*omega_pe/c
    y = all_data['flash','y'][0,target_index:,0]*omega_pe/c

    # Make a directory to hold all of the interpolation objects
    interp_dir = output_dir / "interp"
    if not (interp_dir).exists():
        (interp_dir).mkdir()

    from scipy.interpolate import RegularGridInterpolator
    import pickle
    

    for field in normalizations.keys():
        field_data = np.array(all_data['flash', field][:, target_index:, z_middle_index])/normalizations[field]
        if field == 'edens' or field == 'aldens' or field == 'mgdens':
            np.save(f"{interp_dir}/{field}.npy", field_data)
        else:
            interp1 = RegularGridInterpolator( (y, x), field_data.T, method='linear', bounds_error=False, fill_value=0 )
            with open(f"{interp_dir}/{field}.pkl", "wb") as f:
                pickle.dump(interp1, f)
                f.close()

In [7]:
# This can take a while and always seems to crash the kernel if you try to do more than 1 or 2. Uncomment as needed


# This is normalizing with electron frequency
normalizations = {
    # 'aldens':ne_cgs, # check this normalization
    # 'edens':ne_cgs,
    # 'mgdens':ne_cgs,
    # 'magx':(omega_pe*m_e*c)/e,
    # 'magy':(omega_pe*m_e*c)/e,
    # 'magz':(omega_pe*m_e*c)/e,
    # 'Ex':(omega_pe*m_e*c)/e*alfven_speed/alfven_speed_osiris,
    # 'Ey':(omega_pe*m_e*c)/e*alfven_speed/alfven_speed_osiris, 
    # 'Ez':(omega_pe*m_e*c)/e*alfven_speed/alfven_speed_osiris,
}
save_slice(normalizations)

# This is normalizing with ion frequency
normalizations = {
    # 'aldens':ne_cgs, # check this normalization
    # 'edens':ne_cgs,
    # 'mgdens':ne_cgs,
    # 'magx':((omega_pi*omega_pe/omega_pi_osiris)*m_e*c)/e,
    # 'magy':((omega_pi*omega_pe/omega_pi_osiris)*m_e*c)/e,
    # 'magz':((omega_pi*omega_pe/omega_pi_osiris)*m_e*c)/e,
    # 'Ex':((omega_pi*omega_pe/omega_pi_osiris)*m_e*c**2)/e,
    # 'Ey':((omega_pi*omega_pe/omega_pi_osiris)*m_e*c**2)/e, 
    # 'Ez':((omega_pi*omega_pe/omega_pi_osiris)*m_e*c**2)/e,
}
# save_slice(normalizations)

In [12]:
normalizations = {
    # 'velx':alfven_speed/alfven_speed_osiris,
    # 'vely':alfven_speed/alfven_speed_osiris,
    # 'velz':alfven_speed/alfven_speed_osiris,
    # 'vthele':c, # already normalized
    # 'vthal':c, # already normalized
    # 'vthmg':c, # already normalized
}
save_slice(normalizations)

In [11]:
target_index = 84

x_ion = all_data['flash','x'][:,0,0]*omega_pi/c
print(f"Bounds of simulation in ion inertial length: x {round(x_ion.value[0],1)},{round(x_ion.value[-1],1)} c/omega_pi")
y_ion = all_data['flash','y'][0,target_index:,0]*omega_pi/c
print(f"Bounds of simulation in ion inertial length: y {round(y_ion.value[0],1)},{round(y_ion.value[-1],1)} c/omega_pi")

print()

x = all_data['flash','x'][:,0,0]*omega_pe/c
print(f"Bounds of simulation in osiris units: x {round(x.value[0],0)},{round(x.value[-1],0)} c/omega_pe")
y = all_data['flash','y'][0,target_index:,0]*omega_pe/c
print(f"Bounds of simulation in osiris units: y {round(y.value[0],0)},{round(y.value[-1],0)} c/omega_pe")

print()

x_ion_osiris = all_data['flash','x'][:,0,0]*(omega_pi*omega_pe/omega_pi_osiris)/c
print(f"If we seek to conserve ion inertial length: x {round(x_ion_osiris.value[0],1)},{round(x_ion_osiris.value[-1],1)} c/omega_pe")
y_ion_osiris = all_data['flash','y'][0,target_index:,0]*(omega_pi*omega_pe/omega_pi_osiris)/c
print(f"If we seek to conserve ion inertial length: y {round(y_ion_osiris.value[0],1)},{round(y_ion_osiris.value[-1],1)} c/omega_pe")



Bounds of simulation in ion inertial length: x -27.8,27.8 c/omega_pi
Bounds of simulation in ion inertial length: y 0.4,46.3 c/omega_pi

Bounds of simulation in osiris units: x -2522.0,2522.0 c/omega_pe
Bounds of simulation in osiris units: y 36.0,4206.0 c/omega_pe

If we seek to conserve ion inertial length: x -277.6,277.6 c/omega_pe
If we seek to conserve ion inertial length: y 3.9,462.8 c/omega_pe


In [9]:
from pathlib import Path
import pickle

X1, X2 = np.meshgrid( x, y, indexing='xy' )

# Change this if looking at another simulation
interp_dir = output_dir / "interp"
interp_files = interp_dir.glob("*.pkl")

for f in interp_files:
    name = f.stem
    with open(f, "rb") as f:
        loaded_interpolator = pickle.load(f)
    synthetic = loaded_interpolator((X2,X1))


    plt.imshow(synthetic, origin='lower')
    cbar = plt.colorbar()
    cbar.set_label('Field Value [arb]')
    plt.title(name)
    plt.xlabel("x [c/$\omega_{pe}$]")
    plt.ylabel("y [c/$\omega_{pe}$]")
    plt.show()

interp_files = interp_dir.glob("*.npy")

for f in interp_files:
    name = f.stem
    plt.imshow(np.load(f).T[10:,:], origin='lower')
    cbar = plt.colorbar()
    cbar.set_label('Field Value [arb]')
    plt.title(name)
    plt.xlabel("x [c/$\omega_{pe}$]")
    plt.ylabel("y [c/$\omega_{pe}$]")
    plt.show()

<>:21: SyntaxWarning: invalid escape sequence '\o'
<>:22: SyntaxWarning: invalid escape sequence '\o'
<>:33: SyntaxWarning: invalid escape sequence '\o'
<>:34: SyntaxWarning: invalid escape sequence '\o'
<>:21: SyntaxWarning: invalid escape sequence '\o'
<>:22: SyntaxWarning: invalid escape sequence '\o'
<>:33: SyntaxWarning: invalid escape sequence '\o'
<>:34: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipykernel_750966/2560500566.py:21: SyntaxWarning: invalid escape sequence '\o'
  plt.xlabel("x [c/$\omega_{pe}$]")
/tmp/ipykernel_750966/2560500566.py:22: SyntaxWarning: invalid escape sequence '\o'
  plt.ylabel("y [c/$\omega_{pe}$]")
/tmp/ipykernel_750966/2560500566.py:33: SyntaxWarning: invalid escape sequence '\o'
  plt.xlabel("x [c/$\omega_{pe}$]")
/tmp/ipykernel_750966/2560500566.py:34: SyntaxWarning: invalid escape sequence '\o'
  plt.ylabel("y [c/$\omega_{pe}$]")


NameError: name 'x' is not defined